# CNN pattern model, proposal no.2.1

In [ ]:

# Imports
import init
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
import wandb
import os
from src.loaders.CNNPatt2Dataset import CNNPatt2Dataset
from src.core.config import Conf
from src.models.ModifiedIOULoss import ModifiedDistanceBoxIOULoss, ModifiedCompleteBoxIOULoss
from src.models.train import train
from src.models.validate import PatternValidation
from src.loaders.spectrogram_cacher import SpectrogramCacher

validate_pattern = PatternValidation()

DATA_ROOT = "../kaggle-processed"
assert os.path.exists(DATA_ROOT)

## Config and w&b setup

In [ ]:
config = Conf(
    n_reps_per_epoch=5,
    empty_per_sound_multiplier=0,
    lr=1.e-4,
    augmentation_std=0.3,
    model_type="CNN with improved data augmentation",
    loss_fn="Modified complete IOU loss"
)

wandb.login()
wandb_run = wandb.init(
    project="InzCNNPatternModel",
    notes="IOU error is the most important here",
    config=config.to_dict()
)

## Loading dataset

In [ ]:
# setting up datasets
train_data = CNNPatt2Dataset(os.path.join(DATA_ROOT, "train"), config,
                             seed=42, region_classification=False, augment_gauss_max_std=config.augmentation_std)
valid_data = CNNPatt2Dataset(os.path.join(DATA_ROOT, "valid"), config,
                             seed=42, region_classification=False, augmentation=False)

print(f"Prepared {len(train_data)} training samples, {len(valid_data)} samples")
# set up data loader
train_loader = DataLoader(train_data, batch_size=config.batch_size, shuffle=True, drop_last=True, num_workers=3)
valid_loader = DataLoader(valid_data, batch_size=config.batch_size, shuffle=True, drop_last=True, num_workers=3)

# Caching spectrograms

In [ ]:
files_to_cache = train_data.get_all_files() + valid_data.get_all_files()
sample_cache = SpectrogramCacher.get_instance()
sample_cache.cache_all(files_to_cache, config.to_dict())

# MODEL

In [ ]:
# STAGE 2: narrowing down the selection
torch.random.manual_seed(42)


class CNNPatternNetwork(nn.Module):
    OUTPUT_SHAPE = 2

    def __init__(self, config: Conf):
        super().__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(1, config.cnn_1_filters, kernel_size=(7, 3), padding=(3, 1)), 
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(config.cnn_1_filters, config.cnn_2_filters, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Conv2d(config.cnn_2_filters, config.cnn_3_filters, kernel_size=3, padding=1),
            nn.LeakyReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.flatten = nn.Flatten()
        self.fc_layers = nn.Sequential(
            nn.Linear(config.cnn_3_filters * (config.H // 8) * (config.W // 8), config.fc_size),  # Adjusted to the new output size
            nn.LeakyReLU(),
            nn.Dropout(config.fc_dropout),
            nn.Linear(config.fc_size, config.fc_size),
            nn.LeakyReLU(),
            nn.Dropout(config.fc_dropout),
            nn.Linear(config.fc_size, self.OUTPUT_SHAPE),
            nn.Sigmoid()
        )

    def forward(self, x):
        x = self.conv_layers(x)
        x = self.flatten(x)
        x = self.fc_layers(x)
        return x

model = CNNPatternNetwork(config)

# TRAIN

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=config.lr)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)

loss_fn = ModifiedCompleteBoxIOULoss(config).to(device)

In [ ]:
train(
    config,
    train_loader,
    valid_loader,
    model,
    device,
    loss_fn,
    optimizer,
    validate_pattern,
    wandb_run
)

In [ ]:
wandb_run.finish()